In [1]:
import os
import numpy as np
import pandas as pd
import time
from datetime import datetime

import argparse
from argparse import RawTextHelpFormatter

from tqdm import tqdm

# bag of words imports
from skl_groups.features import Features
from skl_groups.summaries import BagMean
from skl_groups.summaries import BagOfWords
from sklearn.pipeline import Pipeline
from skl_groups.divergences import KNNDivergenceEstimator
from skl_groups.kernels import PairwisePicker, Symmetrize, RBFize, ProjectPSD, FlipPSD, ShiftPSD, SquarePSD
from skl_groups import kernels


import h5py

In [13]:
def calculate_div_matrix(feature_data_path,key_pattern,divFcn,temp_dir,k,hdf5_output_path):

    import logging
    
    logging.basicConfig(filename=hdf5_output_path + '.log',
                        format='%(asctime)s - %(levelname)s - %(message)s',
                        datefmt='%m/%d/%Y %I:%M:%S %p')
    
    logger = logging.getLogger('skl_groups')
    logger.setLevel('INFO')
    logger.addHandler(logging.StreamHandler())
    
    features_df = pd.HDFStore(feature_data_path,mode='r') #h5py.File(feature_data_path, 'r')
    feature_keys = list(features_df.keys())
    #print(feature_keys)
    if key_pattern is not None:
        feature_keys = [key for key in feature_keys if key_pattern in key]
   
    
    bags = [features_df.get(key).to_numpy() for key in feature_keys]

    feats = Features(bags)

    import tempfile
    
    tmpdir = tempfile.mkdtemp(dir=temp_dir)
    logger.info('Temporary directory: {}'.format(tmpdir))
    
    knnDiv = KNNDivergenceEstimator(div_funcs=[divFcn],
                                memory=tmpdir,
                                Ks=[k],  
                                n_jobs=-1)
    print(knnDiv)
    from skl_groups.utils import show_progress
    show_progress('skl_groups.divergences.knn.progress')

    div_mat = knnDiv.fit_transform(feats)
    
    import shutil
    shutil.rmtree(tmpdir)
    features_df.close()
    
    # Reduce matrix dimension to only pairwise similarity one
    div_mat = div_mat[0][0]
    csv_header = []

    for feature_key in feature_keys:
        feature_key = feature_key.replace("/sid_","")
        feature_key = feature_key.replace("/phase1/Haralick_Features","")
        csv_header.append(feature_key)

    div_df = pd.DataFrame(div_mat,index=csv_header,columns=csv_header)
    div_df.to_csv('/pylon5/ac5616p/debdas/SCCORProcessing/tmp/COPDSCCORkl_div_3nn_mat_haralick.csv')  
    #logger.info('Saving a divergence matrix output: {}'.format(hdf5_output_path))
    
    #div_df.to_hdf(hdf5_output_path, key=divFcn + '_' + feature_data_path.replace('.h5',''))
    
    return True

In [14]:
calculate_div_matrix('/pylon5/ac5616p/debdas/SCCORProcessing/COPD_SCCOR_features_db.h5', 'Haralick_Features', 'kl', '/pylon5/ac5616p/debdas/SCCORProcessing/tmp', 3, '/pylon5/ac5616p/debdas/SCCORProcessing/tmp/COPD_SCCORkl_div_3nn_mat.h5')

Temporary directory: /pylon5/ac5616p/debdas/SCCORProcessing/tmp/tmp7tlgs36f
Temporary directory: /pylon5/ac5616p/debdas/SCCORProcessing/tmp/tmp7tlgs36f
Temporary directory: /pylon5/ac5616p/debdas/SCCORProcessing/tmp/tmp7tlgs36f
Temporary directory: /pylon5/ac5616p/debdas/SCCORProcessing/tmp/tmp7tlgs36f
Temporary directory: /pylon5/ac5616p/debdas/SCCORProcessing/tmp/tmp7tlgs36f
Temporary directory: /pylon5/ac5616p/debdas/SCCORProcessing/tmp/tmp7tlgs36f
Building indices...
Building indices...
Building indices...
Building indices...
Building indices...
Building indices...
index building:
index building: |                                              | ETA:  --:--:--
index building: |                                              | ETA:  --:--:--
index building: |                                              | ETA:  --:--:--
index building: |                                              | ETA:  --:--:--
index building: |                                              | ETA:  --:--:--


KNNDivergenceEstimator(Ks=[3], clamp=True, div_funcs=['kl'], do_sym=False,
            flann_algorithm='auto', flann_args=None,
            memory='/pylon5/ac5616p/debdas/SCCORProcessing/tmp/tmp7tlgs36f',
            min_dist=0.001, n_jobs=-1, version='best')


 70 of 70 (100%) |##############################################| Time: 0:00:00

 70 of 70 (100%) |##############################################| Time: 0:00:00

 70 of 70 (100%) |##############################################| Time: 0:00:00

 70 of 70 (100%) |##############################################| Time: 0:00:00

 70 of 70 (100%) |##############################################| Time: 0:00:00

 70 of 70 (100%) |##############################################| Time: 0:00:00

Getting within-bag distances...
Getting within-bag distances...
Getting within-bag distances...
Getting within-bag distances...
Getting within-bag distances...
Getting within-bag distances...
within-bag distances:
within-bag distances:                                          | ETA:  --:--:--
within-bag distances:                                          | ETA:  --:--:--
within-bag distances:                                          | ETA:  --:--:--
within-bag distances:                                       

ValueError: ndarray is not C-contiguous

In [ ]:
/pghbio/dbmi/batmanlab/chirayu/Data/COPDGene/Test_Reproducing_Phase-1_Images/22778J/Phase-1/Isotropic/22778J_INSP_SHARP_UAB_COPD_BSpline_Iso1.0mm.nii.gz

/pghbio/dbmi/batmanlab/chirayu/Data/COPDGene/Images/15567F/Phase-2/Isotropic/15567F_INSP_STD_343_PIT_COPD2_BSpline_Iso1.0mm.nii.gz

/pylon5/ac5616p/debdas/SCCORProcessing/nifti/1-0808-8/Date_20110420/002/002_Iso1.0mm.nii.gz

/pylon5/ac5616p/debdas/SCCORProcessing/nifti/1-0803-7/Date_20110309/002/002_Iso1.0mm.nii.gz

In [13]:
nifti_img_path = '/pghbio/dbmi/batmanlab/chirayu/Data/COPDGene/Test_Reproducing_Phase-1_Images/22778J/Phase-1/Isotropic/22778J_INSP_SHARP_UAB_COPD_BSpline_Iso1.0mm.nii.gz'
file_reader = sitk.ImageFileReader()
file_reader.SetFileName(nifti_img_path)
file_reader.ReadImageInformation()
print('image size: {0}\nimage spacing: {1}'.format(file_reader.GetSize(),
                                                    file_reader.GetSpacing()))

image size: (360, 360, 305)
image spacing: (1.0, 1.0, 1.0)


In [14]:
nifti_img_path = '/pghbio/dbmi/batmanlab/chirayu/Data/COPDGene/Images/15567F/Phase-2/Isotropic/15567F_INSP_STD_343_PIT_COPD2_BSpline_Iso1.0mm.nii.gz'
file_reader = sitk.ImageFileReader()
file_reader.SetFileName(nifti_img_path)
file_reader.ReadImageInformation()
print('image size: {0}\nimage spacing: {1}'.format(file_reader.GetSize(),
                                                    file_reader.GetSpacing()))

image size: (343, 343, 288)
image spacing: (1.0, 1.0, 1.0)


In [15]:
nifti_img_path = '/pylon5/ac5616p/debdas/SCCORProcessing/nifti/1-0808-8/Date_20110420/002/002_Iso1.0mm.nii.gz'
file_reader = sitk.ImageFileReader()
file_reader.SetFileName(nifti_img_path)
file_reader.ReadImageInformation()
print('image size: {0}\nimage spacing: {1}'.format(file_reader.GetSize(),
                                                    file_reader.GetSpacing()))

image size: (291, 291, 316)
image spacing: (1.0, 1.0, 1.0)


In [16]:
nifti_img_path = '/pylon5/ac5616p/debdas/SCCORProcessing/nifti/1-0803-7/Date_20110309/002/002_Iso1.0mm.nii.gz'
file_reader = sitk.ImageFileReader()
file_reader.SetFileName(nifti_img_path)
file_reader.ReadImageInformation()
print('image size: {0}\nimage spacing: {1}'.format(file_reader.GetSize(),
                                                    file_reader.GetSpacing()))

image size: (330, 330, 321)
image spacing: (1.0, 1.0, 1.0)
